In [83]:
import requests as req
import argparse 
import threading
import os
from bs4 import BeautifulSoup
from fpdf import FPDF

In [74]:
iurl = "https://kodansha.us/series/a-silent-voice/"
response = req.get(iurl)

In [75]:
soup = BeautifulSoup(response.text)
mid = soup.find("manga-reader")["chapter-id"]

In [76]:
jurl = "https://kodansha.us/wp-content/themes/kodansha-usa/chapter-json.php"
Image_dir = "images/"
Output_dir = "output/"

In [77]:
payload={'id': mid}
response = req.post(jurl,data=payload)

In [78]:
data = response.json()
name = data["chapter_volume"]
url_list = data['chapter_pages']

In [79]:
chapter_img_path = Image_dir+name+"/"
try:
    os.mkdir(chapter_img_path)
except:
    print("Folder Present")

In [80]:
def download(purl, file_loc):
    img_res = req.get(purl)
    if img_res.status_code == 200:
        with open(file_loc, 'wb') as ifile:
            ifile.write(img_res.content)

In [95]:
def export_pdf(name):
    pdf = FPDF('P','mm','A4')
    x,y,w,h = 0,0,210,297
    chapter_img_path = Image_dir+name+"/"
    img_list = os.listdir(chapter_img_path)
    img_count = len(img_list)
    for ind in range(img_count):
        page_name = chapter_img_path + str(ind) + ".jpg"
        pdf.add_page()
        pdf.image(page_name,x,y,w,h)

    pdf.output(Output_dir+name+".pdf","F")

In [81]:
for ind,obj in list(enumerate(url_list)):
    purl = obj['url']
    img_path = chapter_img_path + str(ind) + ".jpg"
    download_thread = threading.Thread(target=download, args=(purl,img_path))
    download_thread.start()

export_pdf(name)